# Data Preprocessing
## Importation of the data

In [1]:
import pandas as pd

fdaDF = pd.read_csv('output-files/fda-ADMET-properties.csv')
worldNotFdaDF = pd.read_csv('output-files/world-not-fda-ADMET-properties.csv')
reactiveDF = pd.concat([fdaDF, worldNotFdaDF])

anodyneDF = pd.read_csv('output-files/anodyne_1-1000-ADMET-properties.csv')

A column containing a label for each molecule should be added:

In [2]:
reactiveDF['label'] = 0
anodyneDF['label'] = 1

The anodyne and the reactive molecules datasets should be concatenated:

In [3]:
df = pd.concat([reactiveDF, anodyneDF])

## Visualization of the dataset

In [4]:
df.sample(5)

,Molecule,Canonical SMILES,Formula,MW,#Heavy atoms,#Aromatic heavy atoms,Fraction Csp3,#Rotatable bonds,#H-bond acceptors,#H-bond donors,...,Ghose #violations,Veber #violations,Egan #violations,Muegge #violations,Bioavailability Score,PAINS #alerts,Brenk #alerts,Leadlikeness #violations,Synthetic Accessibility,label
1252,Molecule 53,Fc1ccc(cc1)C(=O)C1CCN(CC1)CCn1c(=O)[nH]c2c(c1=...,C22H22FN3O3,395.43,29,16,0.32,5,5,1,...,0,0,0,0,0.55,0,0,1,3.25,0
5987,Molecule 88,O=C(Nc1ccccc1C)Nc1ccc2c(n1)ccnc2,C16H14N4O,278.31,21,16,0.06,4,3,2,...,0,0,0,0,0.55,0,0,0,2.27,1
6102,Molecule 3,CCC1(CC)SCCNC1=N,C8H16N2S,172.29,11,0,0.88,2,1,2,...,0,0,0,1,0.55,0,1,1,2.55,1
2506,Molecule 7,OC(=O)[C@@]12CCCC([C@@H]2CCc2c1c(O)c(c(c2)C(C)...,C20H28O4,332.43,24,6,0.65,2,4,3,...,0,0,0,0,0.56,1,1,1,3.81,0
9379,Molecule 80,O=C(Nc1ccc(cc1C(=O)O)F)/C=C(\c1ccc2c(c1)cccc2)/C,C21H16FNO3,349.36,26,16,0.05,5,4,2,...,0,0,0,1,0.56,0,1,1,2.60,1


The 'Molecule' column should be removed, it doesn't hold any information, it was used as an index when the data has been retrieved from the web server. The SMILES of the molecules should be stored in another frame for later use if needed.

In [5]:
smiles = df['Canonical SMILES']
df = df.drop(['Molecule', 'Canonical SMILES'], axis=1)

Let's display the number of unique values in each columns along with their data type:

In [6]:
pd.DataFrame.from_records([(col, df[col].nunique(), df[col].dtypes) for col in df.columns],
                          columns=['Column_Name', 'Num_Unique', 'Data_Type']).sort_values(by=['Num_Unique'])

,Column_Name,Num_Unique,Data_Type
47,label,2,int64
28,GI absorption,2,object
29,BBB permeant,2,object
30,Pgp substrate,2,object
31,CYP1A2 inhibitor,2,object
32,CYP2C19 inhibitor,2,object
33,CYP2C9 inhibitor,2,object
34,CYP2D6 inhibitor,2,object
35,CYP3A4 inhibitor,2,object
39,Veber #violations,3,int64


The formulas of the molecules should be removed as there is too much unique values for this variable to be used as it is non numerical:

In [7]:
df = df.drop(['Formula'], axis=1)

The columns with an 'object' data types are possible caterogical variables. Some columns with a numerical could also be used as caterogical variables as they hold only a few unique values. Let's convert the columns holding less than 10 unique values to the 'category' data type:

In [8]:
for col in df.columns:
    if df[col].nunique() < 10 :
        df[col] = df[col].astype('category')

We can visualize an histogram of each columns against the labels:

In [9]:
#%matplotlib inline
#import matplotlib.pyplot as plt

#groupedData = df.groupby('label')

#for n, col in enumerate(df.columns):
#    if col != 'label':
#        plt.subplot(10, 5, n + 1)
#        groupedData[col].hist(figsize=(40, 40), alpha=0.4, bins=50)
#        plt.title(col, fontsize=20)

Let's normalize the numerical data using min max normalization:

In [10]:
for col in df._get_numeric_data():
    df[col] = ((df[col] - df[col].min()) / (df[col].max() - df[col].min()))

Let's encode the caterogical variables using One-Hot-Enconding:

In [11]:
cols = df.select_dtypes(include=['category']).columns
for col in cols:
    df[col] = df[col].cat.codes
cols = [col for col in cols if df[col].nunique() > 2]
df = pd.get_dummies(df, columns=cols, prefix=cols)

Finally, let's separate the labels from the dataset:

In [12]:
y = df['label']
X = df.drop(['label'], axis=1)

# Training and evaluating the model

We are using a random forest classifier, let's take a look on the available parameters and their default values from the random forest classifier of the scikit-learn library:

In [13]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
print('+' + '-' * 25 + '+' + '-' * 8 + '+')
for param, value in clf.get_params().items():
    print('|{:25s}|{:8s}|'.format(param, str(value)))
    print('+' + '-' * 25 + '+' + '-' * 8 + '+')

+-------------------------+--------+
|bootstrap                |True    |
+-------------------------+--------+
|class_weight             |None    |
+-------------------------+--------+
|criterion                |gini    |
+-------------------------+--------+
|max_depth                |None    |
+-------------------------+--------+
|max_features             |auto    |
+-------------------------+--------+
|max_leaf_nodes           |None    |
+-------------------------+--------+
|min_impurity_decrease    |0.0     |
+-------------------------+--------+
|min_impurity_split       |None    |
+-------------------------+--------+
|min_samples_leaf         |1       |
+-------------------------+--------+
|min_samples_split        |2       |
+-------------------------+--------+
|min_weight_fraction_leaf |0.0     |
+-------------------------+--------+
|n_estimators             |warn    |
+-------------------------+--------+
|n_jobs                   |None    |
+-------------------------+--------+
|

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
%matplotlib inline
import matplotlib.pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y)

scores_train = []
scores_test = []
for n in range(0, 500, 10):
    clf = RandomForestClassifier(n_estimators=n)
    clf.fit(X_train, y_train)
    
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
    
    scores_train += [mean_squared_error(y_train, y_pred_train)]
    scores_test += [mean_squared_error(y_test, y_pred_test)]

ValueError: n_estimators must be greater than zero, got 0.

In [ ]:
plt.plot(range(10, 500, 10), scores_train, c='r')
plt.plot(range(10, 500, 10), scores_test, c='b')

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

clf = RandomForestClassifier(n_estimators=10)
skf = StratifiedKFold(n_splits=10, shuffle=True)
cross_val_score(clf, X, y, cv=skf)

In [ ]:
clf.fit(X, y)
dico = dict(zip(X.columns, clf.feature_importances_))

print('+' + '-' * 30 + '+' + '-' * 20 + '+')
for col, val in sorted(dico.items(), key=lambda x: x[1]):
    print('|{:30s}|{:20f}|'.format(col, val))
    print('+' + '-' * 30 + '+' + '-' * 20 + '+')